In [ ]:
pip install gradio transformers

In [2]:
import gradio as gr
from transformers import AutoModelWithLMHead,  AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
basemodel = AutoModelWithLMHead.from_pretrained('SWQ/gpt2-medium-combine')
basetokenizer = AutoTokenizer.from_pretrained('gpt2')

/usr/local/lib/python3.8/dist-packages/transformers/models/auto/modeling_auto.py:1177: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Downloading:   0%|          | 0.00/967 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
t5model = AutoModelForSeq2SeqLM.from_pretrained("Luffyt/t5-base-gec-combine_data")
t5tokenizer = AutoTokenizer.from_pretrained("Luffyt/t5-base-gec-combine_data")

Downloading:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [6]:
def greet(sentence):

    def base_grammar_correct(model, tokenizer, text):
      input_ids = tokenizer("<s>"+text+"</s>>>>><p>", return_tensors="pt")
      outputs = model.generate(**input_ids, num_beams=3, num_return_sequences=2,pad_token_id = 50256, max_new_tokens =  len(input_ids.input_ids[0])-3)
      return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    def t5_grammar_correct(model, tokenizer, text):
      input_ids = tokenizer([text], return_tensors="pt", truncation=True, max_length=64)
      outputs = model.generate(**input_ids, num_beams=3, num_return_sequences=2, max_length=64)
      return tokenizer.batch_decode(outputs, skip_special_tokens=True)

    baseresult = base_grammar_correct(basemodel, basetokenizer, sentence)
    t5result = t5_grammar_correct(t5model, t5tokenizer, sentence)

    return baseresult[0].split('</s>>>>><p>')[1].split('</p>')[0],baseresult[1].split('</s>>>>><p>')[1].split('</p>')[0],t5result[0],t5result[1]


demo = gr.Interface(fn=greet, 
                    inputs=gr.Textbox(lines=2, placeholder="type sentences need to be corrected"), 
                    outputs=[gr.Textbox(label='baseline model suggestion 1'),gr.Textbox(label='baseline model suggestion 2'),
                    gr.Textbox(label='t5 model suggestion 1'),gr.Textbox(label='t5 model suggestion 2')])

demo.launch(share=True)   

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://26403c67eb1d323f.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
